# Initial settings

<div class="alert alert-block alert-info">Badlands imports a regular grid that is later triangularized and used to compute erosion and deposition induced by a combination of climate, tectonics and sea-level changes.</div>

In this example, we will evaluate the effects of the **Stream Power Law** and the **Tool & Cover** on landscape dissection. 

We will conduct two numerical simulations of the dissection of an initial subdued landscape for these two cases. We will then track the evolution of erosion rate together with the evolution of topography, summarized by changes in the minimum, mean, and highest elevations of two diagnostic areas.

In this first notebook we will build our synthetic landscape and the displacement file.

***

This is done in 2 steps:

+ First we create a synthetic landscape composed of a 150 x 50 km plateau dotted with one 10 km in diameter relief with an elevation of 1,000 m. These residual relief is placed on the main drainage divide. 

+ Then this landscape experiences uplift at a rate of 200m/My during the first 5 My (cumulative uplift for the region 1000 m). We will then simulate a 5 My-long topographic decay. 

<img src="images/setting.jpg" alt="geometry" width="800" height="400"/>

In [ ]:
%matplotlib inline

import numpy as np

# Import badlands grid generation toolbox
import badlands_companion.toolGeo as simple
# Import badlands displacement map generation toolbox
import badlands_companion.toolTec as tec

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

## Initial paleotopography

We use the companion to create a half-ellipsoid (**dome**-like) surface. The ellipsoid equation center on ($x_o,y_o,z_o$) is of the form:

$$ \frac{(x-x_o)^2}{a^2} + \frac{(y-y_o)^2}{b^2} + \frac{(z-z_o)^2}{c^2} = 1 $$

where $a,b,c$ are strictly positives and equal to the lenght of the ellipsoid half axes.

We first initialize the extent of the simulation area.

The function takes 3 arguments:
+ the extent of the domain along the X-axis [xmin,xmax] in metres,
+ the extent of the domain along the Y-axis [ymin,ymax] in metres,
+ the grid spacing `dx` in metres.

For more information regarding the function uncomment the following cell.

In [ ]:
#help(simple.toolGeo.__init__)

In [ ]:
dome = simple.toolGeo(extentX=[0.,150000.], extentY=[0.,50000.], dx=500.)

We create the dome wave by defining 6 parameters:

+ the ellipsoif half axes: {a,b,c} in metres,
+ the baselevel of the simulatin grid ($z_o$) in metres,
+ the center of the dome {x,y} coordinates in metres.


The function `buildDome` solves for the given parameters the following equation for the elevation:

$$ z = z_o + c \sqrt{1 - \frac{(x-x_o)^2}{a^2} -\frac{(y-y_o)^2}{b^2} } $$

For more information regarding the function uncomment the following cell.


In [ ]:
#help(dome.buildDome)

In [ ]:
dome.Z = dome.buildDome(a=5000., b=5000., c=1000., base=0., xcenter=75000., ycenter=25000.)

It is possible to have a look at the grid surface using plotly library before proceeding to the creation of the badlands surface.

In [ ]:
#help(dome.viewGrid)

In [ ]:
dome.viewGrid(width=700, height=700, zmin=-800, zmax=2200, zData=dome.Z, title='Export Dome Grid')

<div class="alert alert-block alert-info">Beware the distortion of the mesh along the x-axis makes our shape looks strange but this is actually a _plotly_ problem and not an issue in the definition of our initial topography!</div>

Once the surface has been generated, we export a **CSV** regular grid that will be loaded in the simulation. The file provides for each line the following information:
+ X coordinates in meters (this axis has a West to East orientation),
+ Y coordinates in meters (this axis has a South to West orientation), 
+ Z coordinates in meters.

**Note:** Nodes must be defined in increasing order from the South/West corner, first along the X axis.

The `buildGrid` function takes the return elevation from the `buildDome` function and creates the badlands grid in the appropriate format.

For more information regarding the function uncomment the following cell.

In [ ]:
#help(dome.buildGrid)

In [ ]:
dome.buildGrid(elevation=dome.Z, nameCSV='data/nodes')

## Tectonic setting

For our simulations we will generate a uniform vertical displacements (uplift) over the regular topographic grid.

The displacement **CSV** file required by Badlands should provide the **cumulative vertical changes** through the simulated period for each node of the regular grid (in meters). 

For a given simulation, it is possible to upload several maps corresponding to different tectonic history. For each of these maps you will need to define the starting and ending time in the XmL input file.

**Attention:** the file defines the *cumulative* displacement **and not** the displacement *rate*.

**Note:** the nodes are read in increasing order based on the regular grid discretization starting from the South-West corner and going first along the X axis (West to East).

Again to create this tectonic file, we also use the companion.

+ First, we set up the dimension of the domain (similar to what is done for the topography grid)
+ Then, we define the displacement values, with a cumulative uplift for the region of 1000 m. 

In [ ]:
dispclass = tec.toolTec(extentX=[0.,150000.], extentY=[0.,50000.], dx=500.)

The displacement is then defined using the `dispclass.disp` variable. We first fill the array with the desired uplift value:

In [ ]:
dispclass.disp = np.zeros(dispclass.Xgrid.shape)
dispclass.disp.fill(1000.)

We now impose a 0 displacement along the borders of the region:

In [ ]:
dispclass.disp[0,:] = 0.
dispclass.disp[-1,:] = 0.
dispclass.disp[:,0] = 0.
dispclass.disp[:,-1] = 0.

In [ ]:
dispclass.dispView(width=600, height=600, dispmin=0, dispmax=1000, dispData=dispclass.disp, title='Export Grid')

Ok sounds all right, now we wrote the displacement file in the `data` folder:

In [ ]:
dispclass.dispGrid(disp=dispclass.disp, nameCSV='data/displacement')